In [1]:
import yaml
import tensorflow as tf
import sys
import numpy as np
from sklearn import datasets
from random import shuffle
from tensorflow.keras.mixed_precision.experimental import Policy

from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model

sys.path.insert(0, '/home/luis/Paquetes')
from modelzoo.common.tf.estimator.run_config import CSRunConfig
from modelzoo.common.tf.estimator.cs_estimator import CerebrasEstimator
from modelzoo.common.tf.estimator.cs_estimator_spec import CSEstimatorSpec

from modelzoo.common.tf.run_utils import (
    check_env,
    get_csrunconfig_dict,
    is_cs,
    save_params,
    update_params_from_args,
)

In [2]:
param_repo_path = '/home/luis/Paquetes/modelzoo/fc_mnist/tf/configs/params.yaml'
with open(param_repo_path, 'r') as fobj:
    p = yaml.safe_load(fobj)

In [3]:
runconfig_params = p['runconfig']
csrunconfig_dict = get_csrunconfig_dict(runconfig_params)
csrunconfig_dict

{'save_summary_steps': 500,
 'save_checkpoints_steps': 10000,
 'keep_checkpoint_max': 2,
 'train_distribute': None,
 'eval_distribute': None,
 'device_fn': None}

In [75]:
tds = tf.data.Dataset.from_tensor_slices(
            np.array([[[5.0, 3.4, 1.6, 0.7],], [[5.0, 3.4, 1.6, 0.7],]]))
list(tds.as_numpy_iterator())

[array([[5. , 3.4, 1.6, 0.7]]), array([[5. , 3.4, 1.6, 0.7]])]

In [82]:
def input_fn(params, mode=tf.estimator.ModeKeys.TRAIN):
    print(f'mode in input_fn is {mode}')
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.data.Dataset.from_tensor_slices(
            np.array([[[5.0, 3.4, 1.6, 0.7],], [[5.0, 3.4, 1.6, 0.7],]]))
    ir = datasets.load_iris()
    #dtype = Policy('mixed_float16', loss_scale=None)
    ds = tf.data.Dataset.from_tensor_slices(
        (ir.data.astype('float16'), ir.target))
        #datasets.load_iris(return_X_y=True))
        
    ds = ds.shuffle(1000).repeat().batch(100, drop_remainder=True)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

def build_model(features, labels, mode, params):
    print(f'mode in build_model is {mode}')
    dtype = Policy('mixed_float16', loss_scale=None)
    tf.keras.mixed_precision.experimental.set_policy(dtype)
    tf.keras.backend.set_floatx('float16')
    i = features
    x = Dense(64, activation='relu',dtype=dtype)(i)
    x = Dense(3, activation='softmax',dtype=dtype)(x) # LOGITS
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return None, x
    
    #loss_per_sample = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=x)
    #loss_per_sample = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=x)
    #loss = tf.reduce_mean(input_tensor=loss_per_sample)
   # loss = tf.keras.losses.SparseCategoricalCrossentropy()
   # tf.compat.v1.summary.scalar('loss', loss)
    lo = tf.keras.losses.SparseCategoricalCrossentropy()
    loss = lo(labels, x)
    return loss, x

def model_fn(features, labels, mode, params):
    print(f'mode in model_fn is {mode}')
    loss, logits = build_model(features, labels, mode, {'p': 2})

    #def model_fn(features, labels, mode, params):
    opt = tf.compat.v1.train.AdamOptimizer()
    train_op = opt.minimize(loss,
            global_step=tf.compat.v1.train.get_or_create_global_step())
    mode = tf.estimator.ModeKeys.TRAIN
    espec = CSEstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            host_call=None)
    return espec

In [89]:
config = CSRunConfig(cs_ip='127.0.0.1', save_checkpoints_steps=1000, log_step_count_steps=1000)

In [90]:
config.matching.autogen_policy = None

AttributeError: 'CSRunConfig' object has no attribute 'matching'

In [84]:
cla = tf.estimator.Estimator(model_dir = '/home/luis/rm_me_estimator',
                      model_fn=model_fn,
                      params=p)
cla.train(input_fn=input_fn, steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/luis/rm_me_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
mode in input_fn is train
INFO:tensorflow:Calling model_fn.
mode in model_fn is train
mode in build_model i

In [85]:
cla.evaluate(input_fn=input_fn, steps=100)

mode in input_fn is eval
INFO:tensorflow:Calling model_fn.
mode in model_fn is eval
mode in build_model is eval
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-02-23T00:59:59Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/luis/rm_me_estimator/model.ckpt-10100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Inference Time : 0.13343s
INFO:tensorflow:Finished evaluation at 2022-02-23-00:59:59
INFO:tensorflow:Saving dict for global step 10100: global_step = 10100, loss = 0.04104805
INFO:tensorflow:Saving 'checkpo

{'loss': 0.04104805, 'global_step': 10100}

In [86]:
for pred in cla.predict(input_fn=input_fn):
    cnt = 0 
    for key, value in pred.items():
        print(f"{key} is {value}")
    break

mode in input_fn is infer
INFO:tensorflow:Calling model_fn.
mode in model_fn is infer
mode in build_model is infer


AttributeError: 'NoneType' object has no attribute 'dtype'

In [87]:
next(cla.predict(input_fn))

mode in input_fn is infer
INFO:tensorflow:Calling model_fn.
mode in model_fn is infer
mode in build_model is infer


AttributeError: 'NoneType' object has no attribute 'dtype'

In [88]:
tf.reduce_mean?

Signature: tf.reduce_mean(input_tensor, axis=None, keepdims=False, name=None)
Docstring:
Computes the mean of elements across dimensions of a tensor.

Reduces `input_tensor` along the dimensions given in `axis` by computing the
mean of elements across the dimensions in `axis`.
Unless `keepdims` is true, the rank of the tensor is reduced by 1 for each
entry in `axis`. If `keepdims` is true, the reduced dimensions are retained
with length 1.

If `axis` is None, all dimensions are reduced, and a tensor with a single
element is returned.

For example:

>>> x = tf.constant([[1., 1.], [2., 2.]])
>>> tf.reduce_mean(x)
<tf.Tensor: shape=(), dtype=float32, numpy=1.5>
>>> tf.reduce_mean(x, 0)
<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.5, 1.5], dtype=float32)>
>>> tf.reduce_mean(x, 1)
<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>

Args:
  input_tensor: The tensor to reduce. Should have numeric type.
  axis: The dimensions to reduce. If `None` (the default